In [11]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
from random import randint 
from sklearn.cluster import KMeans
from sklearn import model_selection
from sklearn import model_selection
from sklearn import metrics

Let's try to predict locations for new hospitals, animal shelters, fire dept.

# Load data
Clean it and save the cleaned version.

In [5]:
calls = pd.read_csv('../data/geoloc_data/app_public_safety/callsForServiceUpdated.csv')
calls = calls[calls.rptonly != True]
calls = calls[calls.cancelled != True]
calls = calls[calls.nature != 'TRANSPORT TO ANIMAL SHELTER']
calls['calltime'] = pd.to_datetime(calls.calltime)
calls['year'] = calls.calltime.apply(lambda x: x.date().year)
calls['month'] = calls.calltime.apply(lambda x: x.date().month)
calls['dayOfMonth'] = calls.calltime.apply(lambda x: x.date().day)
calls['hour'] = calls.calltime.apply(lambda x: x.time().hour)
calls = calls.rename(columns={'calldow' : 'dayOfWeek'})
calls = calls.drop(columns = ['timeclose', 'calltime'], axis = 1)
calls = calls.drop(columns = ['statbeat','ra'])
calls['actualnature'] = [x['nature2'] if x['nature'] is np.nan and x['nature']!=x['nature2']
                        else x['nature'] for i,x in calls.iterrows()]
calls = calls.drop(columns = ['nature','nature2'])
calls = calls.rename(columns={'actualnature' : 'nature'})

calls.to_csv('../data/geoloc_data/app_public_safety/cleaned_callsForServiceUpdated.csv', 
             sep='\t', index=False)

/home/oana/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Let's split the data into 4 data sets based on the 4 agencies.
We will predict locations for 2 versions of the data: the raw one and the filtered one that keepds only highest priority calls.

In [23]:
filtered_calls = pd.read_csv('../data/geoloc_data/app_public_safety/cleaned_callsForServiceUpdated.csv',
                             sep='\t')
geoloc = pd.DataFrame()
geoloc['long'] = filtered_calls['long']
geoloc['lat'] = filtered_calls['lat']
geoloc['agency'] = filtered_calls['agency']
ems_calls = geoloc[geoloc.agency == 'EMS']
ems_calls.drop(columns='agency', inplace=True)
aco_calls = geoloc[geoloc.agency == 'ACO']
aco_calls.drop(columns='agency', inplace=True)
gcsd_calls = geoloc[geoloc.agency == 'GCSD']
gcsd_calls.drop(columns='agency', inplace=True)
gcf_calls = geoloc[geoloc.agency == 'GCF']
gcf_calls.drop(columns='agency', inplace=True)

calls_highestP = filtered_calls[filtered_calls.priority == 'P']
geoloc_highestP = pd.DataFrame()
geoloc_highestP['long'] = calls_highestP['long']
geoloc_highestP['lat'] = calls_highestP['lat']
geoloc_highestP['agency'] = calls_highestP['agency']
ems_calls_highestP = geoloc_highestP[geoloc_highestP.agency == 'EMS']
ems_calls_highestP.drop(columns='agency', inplace=True)
aco_calls_highestP = geoloc_highestP[geoloc_highestP.agency == 'ACO']
aco_calls_highestP.drop(columns='agency', inplace=True)
gcsd_calls_highestP = geoloc_highestP[geoloc_highestP.agency == 'GCSD']
gcsd_calls_highestP.drop(columns='agency', inplace=True)
gcf_calls_highestP = geoloc_highestP[geoloc_highestP.agency == 'GCF']
gcf_calls_highestP.drop(columns='agency', inplace=True)

/home/oana/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/oana/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


## Predict new locations for hospitals, fire dept, animal shelters
Let's try to predict latitute and longitude for next buildings based on the latitude and long of current phone calls.

In [25]:
model = KMeans(n_clusters=1, random_state=123, n_jobs=-1)
model.fit(ems_calls)
print(model.cluster_centers_)
model.fit(ems_calls_highestP)
print(model.cluster_centers_)

[[-79.85061129  36.05415722]]
[[-79.84560087  36.05486576]]


In [26]:
model = KMeans(n_clusters=1, random_state=123, n_jobs=-1)
model.fit(aco_calls)
print(model.cluster_centers_)
model.fit(aco_calls_highestP)
print(model.cluster_centers_)

[[-79.79462101  36.07537658]]
[[-79.80616015  36.07589802]]


In [27]:
model = KMeans(n_clusters=1, random_state=123, n_jobs=-1)
model.fit(gcf_calls)
print(model.cluster_centers_)
model.fit(gcf_calls_highestP)
print(model.cluster_centers_)

[[-79.79368187  36.07173895]]
[[-79.78873048  36.0725713 ]]


In [28]:
model = KMeans(n_clusters=1, random_state=123, n_jobs=-1)
model.fit(gcsd_calls)
print(model.cluster_centers_)
model.fit(gcsd_calls_highestP)
print(model.cluster_centers_)

[[-79.81209641  36.06432225]]
[[-79.80634588  36.06781992]]
